In [ ]:
import langchain


In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv()

In [ ]:
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    raise ValueError("GROQ_API_KEY environment variable not set!")

In [ ]:
os.getenv("GROQ_API_KEY")

### Example 1: Simple LLM Call with streaming


In [ ]:
from langchain.chat_models import init_chat_model
## agr koi b messsage used karne ke liye y used karan pare ga 
from langchain_core.messages import HumanMessage,SystemMessage


In [ ]:
model=init_chat_model("groq:llama-3.1-8b-instant")
model

In [ ]:
## Create messages
message = [ 
    SystemMessage("you are a helpfull AI assistant "),
    HumanMessage("What are the top 2 besnifts of langchain ")
]

In [ ]:
##invoke the model
response=model.invoke(message)
response

In [ ]:
for chunk in model.stream(message):
    print(chunk.content, end="", flush=True)

    ## yh aik alternative method he print karwane k liye 

### Dynamic Prompt Template 
## aik addition prompt de skty hen jo instruction llm ko dena he usko prompt template me dal dety hen 

In [ ]:
from langchain_core.prompts import ChatPromptTemplate 

## create translation app 

translation_template=ChatPromptTemplate.from_messages([
    ("system","You  are a professional translator.Translte the follow text {text} from {source_language} to {target_language}"),
    ("user","{text}")
])

## using the tempalte 
prompt=translation_template.invoke({
    "source_language": "English",
    "target_language": "Russia",
    "text":"LangChain is best for Z generation"
})

In [ ]:
prompt

In [ ]:
translated_response=model.invoke(prompt)
print(translated_response.content)

### Building my First Chain


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain_core.prompts import ChatPromptTemplate 

def create_story_chain():
    ##Template for story generation 
    story_prompt=ChatPromptTemplate.from_messages (
        [
            ("system","You are a creative story teller.Write a short and engaging story based on given theme"),
            ("user","Theme:{theme}\n Setting :{setting}")
        ]
        ## Yh prompt aik model ko jaaiey ga 
    )

    ## Template for story analysis 
    analysis_prompt = ChatPromptTemplate.from_messages(
        [
            ("system","You are literaly best.Analyze th following story and provide insights."),
            ("user","{story}")
        ]
    )
    story_chain=(
        story_prompt| model | StrOutputParser()
        ## pehly story prompt bny ga vh jaoey ga model me or phir stroutputparser me 
        ## stroutparser direcct content le leta he LLM se or vh show karta he
    )

    def analyze_story(story_text):
        return {"story": story_text}
    
    analysis_chain = (
        story_chain
        | RunnableLambda(analyze_story)
        | analysis_prompt
        | model
        | StrOutputParser()
    )

    return analysis_chain



In [ ]:
chain=create_story_chain()
chain

In [ ]:
print(chain.input_schema)

In [ ]:

# Corrected spelling + formatted code
result = chain.invoke({
    "theme": "artificial intelligence",
    "setting": "a futuristic city"
})

print("Story and Analysis:")
print(result)


